<a href="https://colab.research.google.com/github/sid521/ML8-ImageCaptioning/blob/main/prediction_beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist
import matplotlib.image as mpimg
import os
import tensorflow as tf
import cv2
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,GlobalAveragePooling2D,Concatenate, ReLU, LeakyReLU,Reshape, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import metrics
from tensorflow.keras.preprocessing import image
from tensorflow.keras.metrics import categorical_accuracy,top_k_categorical_accuracy
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
import matplotlib.pyplot as plt
import collections
import random
import json
from PIL import Image

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Load Resnet50

In [ ]:
image_features_extract_model = load_model('/content/resnet50_model.hdf5')

In [ ]:
image_features_extract_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
#load word tokenizer
with open('/content/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [ ]:
# load and preprocess image
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224)) # (224,224) is the default input of resnet50
    img = tf.keras.applications.resnet50.preprocess_input(img)
    print(img.shape)
    return img, image_path

## Model

In [ ]:
# Feel free to change these parameters according to your system's configuration
# should be the same as the parameters of the saved model
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
# Shape of the vector extracted from resnet50 is (49, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 49

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 49, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, 49, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 49, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 49, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class CNN_Encoder(tf.keras.Model):
    # This encoder passes those features extracted from resnet50 (49,2048)through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 49, embedding_dim)
        self.fc1 = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc1(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.gru2 = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(self.units)
    self.fc3 = tf.keras.layers.Dense(vocab_size)
    self.drop = tf.keras.layers.Dropout(0.25)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    x= self.drop(x)

    # passing the concatenated vector to the GRU
    output, state = self.gru1(x)
    x = self.drop(output)
    output, state_ = self.gru2(x)
    x = self.drop(output)
    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(x)
    x= self.drop(x)
    x = self.fc2(x)
    x= self.drop(x)
    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))
    # output shape == (batch_size * max_length, vocab)
    x = self.fc3(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [ ]:
len(tokenizer.word_index)
inp = tf.expand_dims([tokenizer.word_index['<start>']], 0)

In [ ]:
# initialise the encoder and decoder
vocab_size=5001
encoder = CNN_Encoder(embedding_dim)
encoder(np.zeros((49,2048)))
encoder.built=True
decoder = RNN_Decoder(embedding_dim, units, vocab_size)
hidden = decoder.reset_state(batch_size=1)
decoder(inp,np.zeros((49,256)),hidden)
decoder.built=True

In [ ]:

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Load model weights

In [ ]:
encoder_path='/content/encoder_gru_stack_2_512_enc_total.h5'
decoder_path='/content/decoder_gru_stack_2_512_enc_total.h5'
encoder.load_weights(encoder_path)
decoder.load_weights(decoder_path)

# Prediction

In [ ]:
# predict the captions using greedy search algorithm
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(
        img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(
            dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions, 1)[0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot
        

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for i in range(len_result):
        temp_att = np.resize(attention_plot[i], (7, 7))
        grid_size = max(np.ceil(len_result/2), 2)
        ax = fig.add_subplot(grid_size, grid_size, i+1)
        ax.set_title(result[i])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [ ]:
def beam_search(image, beam_index=2):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(
        img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []
    start = [tokenizer.word_index["<start>"]]
    start_word = [[start, 0.0]]
    while len(start_word[0][0]) < max_length:
        temp = []
        for s in start_word:
            dec_input = tf.expand_dims([s[0][-1]],1)
            preds, hidden, attention_weights = decoder(
                dec_input, features, hidden)
            word_preds = np.argsort(preds[0])[-beam_index:]
            # Getting the top <beam_index>(n) predictions and creating a
            # new list so as to put them via the model again
            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                prob += preds[0][w]
                temp.append([next_cap, prob])

        start_word = temp
        # Sorting according to the probabilities
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        # Getting the top words
        start_word = start_word[-beam_index:]

    start_word = start_word[-1][0]
    intermediate_caption = [tokenizer.index_word[i] for i in start_word]
    final_caption = []

    for i in intermediate_caption:
        if i != '<end>':
            final_caption.append(i)
        else:
            break

    final_caption = ' '.join(final_caption[1:])
    return final_caption

In [ ]:
# predict the caption of the image in the given path using greedy search
def greedy_predict(path):
    filename = os.path.abspath(path)
    img = Image.open(path)
    result, attention_plot = evaluate(path)
    res = ' '.join(result)
    result = res.replace('<end>',' ')
    print('Prediction Caption:', result)
    img_ = mpimg.imread(filename)
    plt.imshow(img_)

In [ ]:
# predict the caption of the image in the given path using beam search
def beam_predict(path,beam_index=2):
    filename = os.path.abspath(path)
    img = Image.open(path)
    result = beam_search(path,bean_index)
    print('Prediction Caption:', result)
    img_ = mpimg.imread(filename)
    plt.imshow(img_)

## Test your own images

In [ ]:
path = '/content/test.png'
beam_predict(path,beam_index=2)

In [ ]:
path = '/content/test.png'
greedy_predict(path)

#Video Summarization

### Converting Video to frames

In [ ]:
import cv2
list=[]
# Opens the Video file
cap= cv2.VideoCapture('/content/Samplevideo.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        total= i
        # 'total number of frames-1'  for the video
        print(total)
        break
    file_name_path = '/content/frames/frame' + str(i) + '.jpg' 
    list.append(file_name_path)   
    cv2.imwrite(file_name_path,frame)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

4106


In [ ]:
max_length=52
i=0
predics=[]
# empty list to append the predicted captions for all the frames
while i<total:
  image = list[i]
  result= beam_search(image,2)
  predics.append(result)
  #list where captions associated to all frames of video are stored 
  i=i+60
  #taking 1 frame out of every 60 frames
  #considering 1 frame for every 2 seconds

Streaming output truncated to the last 5000 lines.
tf.Tensor([[4]], shape=(1, 1), dtype=int32)
tf.Tensor([[4]], shape=(1, 1), dtype=int32)
(224, 224, 3)
tf.Tensor([[3]], shape=(1, 1), dtype=int32)
tf.Tensor([[16]], shape=(1, 1), dtype=int32)
tf.Tensor([[2]], shape=(1, 1), dtype=int32)
tf.Tensor([[441]], shape=(1, 1), dtype=int32)
tf.Tensor([[23]], shape=(1, 1), dtype=int32)
tf.Tensor([[8]], shape=(1, 1), dtype=int32)
tf.Tensor([[20]], shape=(1, 1), dtype=int32)
tf.Tensor([[14]], shape=(1, 1), dtype=int32)
tf.Tensor([[2]], shape=(1, 1), dtype=int32)
tf.Tensor([[8]], shape=(1, 1), dtype=int32)
tf.Tensor([[18]], shape=(1, 1), dtype=int32)
tf.Tensor([[2]], shape=(1, 1), dtype=int32)
tf.Tensor([[338]], shape=(1, 1), dtype=int32)
tf.Tensor([[8]], shape=(1, 1), dtype=int32)
tf.Tensor([[4]], shape=(1, 1), dtype=int32)
tf.Tensor([[2]], shape=(1, 1), dtype=int32)
tf.Tensor([[4]], shape=(1, 1), dtype=int32)
tf.Tensor([[24]], shape=(1, 1), dtype=int32)
tf.Tensor([[4]], shape=(1, 1), dtype=int32)
t

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 2.6 MB 62.1 MB/s 
     |████████████████████████████████| 1.2 MB 49.8 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 636 kB 56.2 MB/s 
     |████████████████████████████████| 3.3 MB 62.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=66f43ea8e7d9ec9c8e7ae32ef8f2dd1f6c5d3a25f698fa17a3b31e2b37c21211
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#Remove one of the consecutive image caption greater than certain cosine similarity threshold between the two captions
# threshold value is 0.50
import math
i=0
n=math.floor((total/60)+1)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
while i<n-1:
  sentences=[]
  sentences.append(predics[i])
  if len(predics)>=i+1:
    break
  else:  
    sentences.append(predics[i+1])
  model=SentenceTransformer('bert-base-nli-mean-tokens')
  sentence_embeddings=model.encode(sentences)
  #sentence_embeddings.shape
  cs=cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1])
  if cs>0.50:
    del predics[i]
  else:
    i=i+1  


In [ ]:
#number of sentences removed by threshold
print(n-len(predics))

0


### Summarization with T5 base Transformer

In [ ]:
Text=" "
Text=Text+predics[0]
#concat all sentences in list and convert it into one paragraph
for i in range(1,int(n)):
  Text=Text+"."+predics[i]

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-large")
# initialize the model tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-large")

In [ ]:
#encode the text into tensor of integers using the appropriate tokenizer
inputs = t5_tokenizer.encode("summarize: " + Text, return_tensors="pt",truncation=True)

In [ ]:
# summary output generation in form of tokens
outputs = model.generate(
    inputs, 
    max_length=512, 
    min_length=50, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
print(outputs)

tensor([[    0,     3,     9,   562,    28,     3,     9,   418,     5,     9,
           953,    28,   508,  3196,     5,     9,  1021,  3202,    16,   851,
            13,  2013,  9832,     5,     9,  2335,    16,     3,     9,  2062,
             5,     9,  2335,   838,     3,     9, 18942,     5,     9,  2335,
          4125,   416,    12,     3,     9,  2335,    16,     3,     9,   840,
           562,     5,     9,  2335,  3823,    44,     8,  1861,     5,     9,
          2335,  5119,     3,     9,  6177,     5,     9,  2335,  3823,    30,
             3,     9,  2335,     5,     9,  2335,   464,    30,     3,  4500,
          9658,     7,    16,     8,  1228,     5,     9,   388,    16,     3,
             9,  1001, 12646,    11,  6177,    19,  3609,     3,     9,     3,
         19668,   298,  5119,     3,     9, 18701,     5,   152,   625,   828,
            28,     3,     9, 12522,     1]])


In [ ]:
#Final Summarized output
print(t5_tokenizer.decode(outputs[0]))

<pad> a room with a lot.a table with large windows.a young girl in front of wine glasses.a woman in a restaurant.a woman taking a blender.a woman standing next to a woman in a living room.a woman sitting at the camera.a woman wearing a tie.a woman sitting on a woman.a woman working on hood lounges in the kitchen.a man in a black vest and tie is holding a cigarette while wearing a microphone.an old office with a nap</s>
